In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential

In [3]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rounaksarkar/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
data = pd.read_csv('/Users/rounaksarkar/Desktop/ML:AI/3. Deep Learning Projects/project5: Fake News Classification/train.csv')

In [8]:
data

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [9]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [10]:
data.describe()

,id,label
count,20800.000000,20800.000000
mean,10399.500000,0.500625
std,6004.587135,0.500012
min,0.000000,0.000000
25%,5199.750000,0.000000
50%,10399.500000,1.000000
75%,15599.250000,1.000000
max,20799.000000,1.000000


In [11]:
data.shape

(20800, 5)

In [14]:
data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [15]:
data.dropna()
data.reset_index(inplace=True)

In [17]:
X = data.drop("label",axis=1)
y = data["label"]

In [18]:
print(X.shape)
print(y.shape)

(20800, 5)
(20800,)


In [22]:
messages=X.copy()
ps = PorterStemmer()
corpus = []
for i in range(len(messages)):
    review = str(messages['title'][i]) # convert to string
    review = re.sub("[^a-zA-Z]"," ",review)
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]  # (word not in stopwords) and stem(word)
    review = ' '.join(review)    
    corpus.append(review)


### One Hot Encoding

In [24]:
vocab_size=5000
onehot_repr=[one_hot(words,vocab_size)for words in corpus] 

In [25]:
onehot_repr

[[3619, 1767, 2298, 3573, 733, 4373, 359, 1268, 2845, 598],
 [3053, 2612, 265, 634, 1779, 131, 4568],
 [4735, 1895, 3035, 4055],
 [3744, 488, 1357, 449, 2540, 2483],
 [3458, 1779, 3776, 3760, 723, 3977, 1779, 3091, 4236, 2477],
 [331,
  4020,
  2603,
  3287,
  217,
  3543,
  3030,
  386,
  1874,
  1015,
  3800,
  4431,
  1317,
  4705,
  4568],
 [2286, 2286, 2989, 201, 2788, 2929, 1239, 2835, 4698, 340, 2709, 1253],
 [3943, 1596, 3946, 543, 4952, 1457, 2990, 472, 3723, 939, 1260],
 [1103, 1512, 3306, 1918, 3543, 3280, 709, 2882, 4070, 4688, 3723, 939, 1260],
 [1751, 1200, 3415, 2303, 409, 2207, 3543, 752, 3723, 939, 1260],
 [2248, 4348, 1967, 2069, 2577, 2062, 1852, 2057, 3543, 2871],
 [1420, 4971, 1536, 2306, 1653, 1820, 1240, 2127],
 [4098, 2397, 3094, 4800, 1326, 2598, 3376, 170, 4792, 464, 4182],
 [449, 4387, 733, 2062, 3543, 409],
 [1541, 962, 3969, 2329, 3060, 4335, 1485, 1158, 936],
 [2258, 130, 2559, 3115, 2138, 4726, 1990, 3723, 939, 1260],
 [1157, 2809, 776, 581, 3582, 3723, 9

In [26]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1268 2845  598]
 [   0    0    0 ... 1779  131 4568]
 [   0    0    0 ... 1895 3035 4055]
 ...
 [   0    0    0 ... 3723  939 1260]
 [   0    0    0 ... 1792 1371  845]
 [   0    0    0 ...  246 4496 3036]]


In [27]:
# Creating Model for training set
embedding_vector_features=40
model=Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [28]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [29]:
X_final

array([[   0,    0,    0, ..., 1268, 2845,  598],
       [   0,    0,    0, ..., 1779,  131, 4568],
       [   0,    0,    0, ..., 1895, 3035, 4055],
       ...,
       [   0,    0,    0, ..., 3723,  939, 1260],
       [   0,    0,    0, ..., 1792, 1371,  845],
       [   0,    0,    0, ...,  246, 4496, 3036]], dtype=int32)

In [30]:
y_final

array([1, 0, 1, ..., 0, 1, 1])

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [32]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,batch_size=64)

Epoch 1/100
218/218 [==============================] - 4s 14ms/step - loss: 0.3062 - accuracy: 0.8645 - val_loss: 0.1970 - val_accuracy: 0.9223
Epoch 2/100
218/218 [==============================] - 3s 13ms/step - loss: 0.1282 - accuracy: 0.9498 - val_loss: 0.1907 - val_accuracy: 0.9193
Epoch 3/100
218/218 [==============================] - 3s 14ms/step - loss: 0.0895 - accuracy: 0.9666 - val_loss: 0.2040 - val_accuracy: 0.9276
Epoch 4/100
218/218 [==============================] - 3s 13ms/step - loss: 0.0583 - accuracy: 0.9808 - val_loss: 0.2274 - val_accuracy: 0.9097
Epoch 5/100
218/218 [==============================] - 3s 14ms/step - loss: 0.0362 - accuracy: 0.9883 - val_loss: 0.3242 - val_accuracy: 0.9213
Epoch 6/100
218/218 [==============================] - 3s 14ms/step - loss: 0.0216 - accuracy: 0.9940 - val_loss: 0.3251 - val_accuracy: 0.9159
Epoch 7/100
218/218 [==============================] - 3s 14ms/step - loss: 0.0125 - accuracy: 0.9966 - val_loss: 0.3192 - val_accuracy:

In [33]:
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).astype(int)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

215/215 [==============================] - 1s 3ms/step


array([[3144,  305],
       [ 307, 3108]])

In [34]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9108391608391608